In [2]:
import joblib
import pandas as pd

# Load the joblib file
data = joblib.load(r'C:\Users\Dwaipayan C\Downloads\Monthly_Income_Estimation_Target_Encoded_Artifacts_companytrgencode.joblib')

# Ensure that the loaded data is a DataFrame
if isinstance(data, pd.DataFrame):
    # Save the DataFrame to a CSV file
    data.to_csv('output_file_name.csv', index=False)
    print("Data has been successfully converted to CSV.")
else:
    print("Loaded data is not a DataFrame. Please ensure your joblib file contains a DataFrame.")


Data has been successfully converted to CSV.


In [3]:
import joblib
import pandas as pd
import zipfile

# Load the joblib file
data = joblib.load(r'C:\Users\Dwaipayan C\Downloads\Monthly_Income_Estimation_Target_Encoded_Artifacts_companytrgencode.joblib')

# Ensure that the loaded data is a DataFrame
if isinstance(data, pd.DataFrame):
    # Save the DataFrame to a CSV file
    csv_file = 'output_file_name.csv'
    data.to_csv(csv_file, index=False)
    
    # Compress the CSV file into a ZIP file
    zip_file = 'output_file_name.zip'
    with zipfile.ZipFile(zip_file, 'w', zipfile.ZIP_DEFLATED) as z:
        z.write(csv_file)
    
    print("Data has been successfully converted to CSV and compressed into a ZIP file.")
else:
    print("Loaded data is not a DataFrame. Please ensure your joblib file contains a DataFrame.")


Data has been successfully converted to CSV and compressed into a ZIP file.


# Step1

In [20]:
import pandas as pd

# Load your data
df = pd.read_csv(r'C:\Users\Dwaipayan C\OneDrive\Learning\Projects\Income_Estimation\output_file_name.csv')

# Preview the data
print(df.head())


    global_mean  smoothing_factor       loan_company_name  \
0  30892.506245                15                     NaN   
1  30892.506245                15  Expert Security Agency   
2  30892.506245                15  leekie enterprises inc   
3  30892.506245                15          Mangiboa Store   
4  30892.506245                15           Fruits Vendor   

   freq_encodedcompanyName  category_mean_companyName  \
0                      NaN                        NaN   
1                      1.0               25000.000000   
2                      1.0               28000.000000   
3                      1.0               15000.000000   
4                      3.0               28333.333333   

   category_size_companyName  target_encodedcompanyName  \
0                        NaN               30892.506245   
1                        1.0               30524.224605   
2                        1.0               30711.724605   
3                        1.0               29899.22460

In [7]:
len(df['loan_company_name'].drop_duplicates().unique())

64771

# Step2

In [8]:
import re

# Function to clean company names
def preprocess_name(name):
    name = str(name).lower()  # Convert to lowercase
    name = re.sub(r'[^\w\s]', '', name)  # Remove punctuation
    # Remove common suffixes
    suffixes = ['inc', 'incorporated', 'corp', 'corporation', 'ltd', 'limited', 'co', 'company', 'plc']
    pattern = r'\b(?:{})\b'.format('|'.join(suffixes))
    name = re.sub(pattern, '', name)
    name = ' '.join(name.split())  # Remove extra spaces
    return name

# Apply preprocessing
df['clean_name_re'] = df['loan_company_name'].apply(preprocess_name)

# Preview cleaned names
print(df[['loan_company_name', 'clean_name_re']].head())


        loan_company_name           clean_name_re
0                     NaN                     nan
1  Expert Security Agency  expert security agency
2  leekie enterprises inc      leekie enterprises
3          Mangiboa Store          mangiboa store
4           Fruits Vendor           fruits vendor


# Step3 - Using rapidfuzz

In [9]:
import pandas as pd
import re
from rapidfuzz import process, fuzz

# Function to clean company names
def preprocess_name(name):
    name = str(name).lower()  # Convert to lowercase
    name = re.sub(r'[^\w\s]', '', name)  # Remove punctuation
    # Remove common suffixes
    suffixes = ['inc', 'incorporated', 'corp', 'corporation', 'ltd', 'limited', 'co', 'company', 'plc']
    pattern = r'\b(?:{})\b'.format('|'.join(suffixes))
    name = re.sub(pattern, '', name)
    name = ' '.join(name.split())  # Remove extra spaces
    return name

# Load your data

df = pd.read_csv(r'C:\Users\Dwaipayan C\OneDrive\Learning\Projects\Income_Estimation\output_file_name.csv')
# Apply preprocessing
df['clean_name_rapidfuzz'] = df['loan_company_name'].apply(preprocess_name)

# Get a list of unique cleaned names
unique_names = df['clean_name_rapidfuzz'].unique()

# Create a dictionary to map similar names
name_mapping = {}

for name in unique_names:
    if name in name_mapping.values():
        continue  # Skip if already mapped
    # Find matches with a similarity score above the threshold
    matches = process.extract(
        name,
        unique_names,
        scorer=fuzz.token_sort_ratio,  # Use fuzz.token_sort_ratio here
        score_cutoff=90  # Adjust this threshold as needed
    )
    # Map similar names to the representative name
    for match_name, score, _ in matches:
        name_mapping[match_name] = name

# Map the standardized names back to the DataFrame
df['standardized_name'] = df['clean_name_rapidfuzz'].map(name_mapping)

# Optionally, format the standardized names
df['rapidfuzzname'] = df['standardized_name'].str.title()

# Save the cleaned data to a new CSV file
df.to_csv('rapidfuzzname_companyName.csv', index=False)

print("Company names have been standardized and saved to 'rapidfuzzname_companyName.csv'.")


Company names have been standardized and saved to 'rapidfuzzname_companyName.csv'.


# Using jellyfish for Phonetic

In [10]:
import pandas as pd
import jellyfish

# Function to get phonetic code
def get_phonetic_code(name):
    name = str(name)
    code = jellyfish.metaphone(name)
    return code

# Apply phonetic coding
df['phonetic_code'] = df['loan_company_name'].apply(get_phonetic_code)

# Map standardized names based on phonetic codes
df['standardized_name_jellyfish'] = df.groupby('phonetic_code')['loan_company_name'].transform('first')

# Preview the results
print(df[['loan_company_name', 'standardized_name_jellyfish']].head())


        loan_company_name standardized_name_jellyfish
0                     NaN                       Noona
1  Expert Security Agency      Expert Security Agency
2  leekie enterprises inc      leekie enterprises inc
3          Mangiboa Store              Mangiboa Store
4           Fruits Vendor               Fruits Vendor


In [11]:
# Save the cleaned data to a new csv file

df.to_csv("Phonetic_cleaned_company_names.csv", index=False)

# Untilizing Hashing of n-gram

In [12]:
import pandas as pd
import hashlib

# Function to generate n-gram hash
def ngram_hash(name, n=3):
    name = str(name).lower()
    name = re.sub(r'[^\w\s]', '', name)
    tokens = name.split()
    ngrams = [''.join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]
    combined = ' '.join(ngrams)
    return hashlib.md5(combined.encode()).hexdigest()

# Apply n-gram hashing
df['ngram_hash'] = df['loan_company_name'].apply(ngram_hash)

# Map standardized names
df['standardized_name_ngram_hash'] = df.groupby('ngram_hash')['loan_company_name'].transform('first')

# Preview the results
print(df[['loan_company_name', 'standardized_name_ngram_hash']].head())


        loan_company_name standardized_name_ngram_hash
0                     NaN               Mangiboa Store
1  Expert Security Agency       Expert Security Agency
2  leekie enterprises inc       leekie enterprises inc
3          Mangiboa Store               Mangiboa Store
4           Fruits Vendor               Mangiboa Store


In [13]:
df.to_csv("Ngram_cleaned_company_names.csv", index=False)

# Implementing Exact Matching with Extensive Normalization

In [14]:
import pandas as pd
import unicodedata

# Function to perform aggressive normalization
def aggressive_normalize(name):
    name = str(name).lower()
    name = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')
    name = re.sub(r'[^\w\s]', '', name)
    name = re.sub(r'\d+', '', name)  # Remove digits
    stop_words = ['the', 'and', 'of', 'in', 'a']
    tokens = [word for word in name.split() if word not in stop_words]
    suffixes = ['inc', 'incorporated', 'corp', 'corporation', 'company', 'co', 'ltd', 'limited', 'plc']
    tokens = [word for word in tokens if word not in suffixes]
    name = ' '.join(sorted(tokens))
    return name.strip()

# Apply aggressive normalization
df['standardized_name_aggressive_normalize'] = df['loan_company_name'].apply(aggressive_normalize)

# Preview the results
print(df[['loan_company_name', 'standardized_name_aggressive_normalize']].head())


        loan_company_name standardized_name_aggressive_normalize
0                     NaN                                    nan
1  Expert Security Agency                 agency expert security
2  leekie enterprises inc                     enterprises leekie
3          Mangiboa Store                         mangiboa store
4           Fruits Vendor                          fruits vendor


In [16]:
df.to_csv("Aggressive_cleaned_company_names.csv", index=False)

# 5. Clustering with String Similarity

In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN

# Preprocess names
def preprocess(name):
    name = str(name).lower()
    name = re.sub(r'[^\w\s]', '', name)
    return name

df['clean_name_dbscan'] = df['loan_company_name'].apply(preprocess)

# Vectorize names
vectorizer = TfidfVectorizer(min_df=1, analyzer='char', ngram_range=(2,3))
tfidf_matrix = vectorizer.fit_transform(df['clean_name_dbscan'])

# Cluster names using DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=2, metric='cosine')
dbscan.fit(tfidf_matrix)

# Assign cluster labels
df['cluster'] = dbscan.labels_

# Map standardized names
df['standardized_name_dbscan'] = df.groupby('cluster')['loan_company_name'].transform('first')

# Preview the results
print(df[['loan_company_name', 'standardized_name_dbscan']].head())


        loan_company_name standardized_name_dbscan
0                     NaN                     None
1  Expert Security Agency   Expert Security Agency
2  leekie enterprises inc   Expert Security Agency
3          Mangiboa Store   Expert Security Agency
4           Fruits Vendor   Expert Security Agency


In [18]:
df.to_csv("DBSCAN_cleaned_company_names.csv", index=False)

**Final Recommendation**

Given your need for speed and efficiency, I suggest starting with extensive normalization (Option 1) and only moving to clustering methods if necessary. Aggressive preprocessing often resolves most of the standardization issues without heavy computational costs.

**Additional Thoughts**

`Manual Inspection:` After applying automated methods, it's helpful to review a sample to ensure accuracy.

`Hybrid Approach:` Combine methods for better results (e.g., normalization followed by phonetic coding).

`Data Quality Feedback Loop:` As you identify recurring issues, update your preprocessing functions to handle them.

**Sample Workflow**

`Normalize Names Aggressively:` Clean and standardize as much as possible.

`Apply Phonetic Encoding (Optional):` If needed, to catch phonetically similar names.

`Cluster Remaining Variations:` Use clustering for names that are still not matching.

`Validate Results:` Check samples to ensure that unrelated companies aren't merged.

`Iterate and Refine:` Adjust parameters and methods based on findings.



# Claude code for pipeline

In [21]:
import pandas as pd
import re
import jellyfish
import hashlib
import unicodedata
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class CompanyNameStandardizer(BaseEstimator, TransformerMixin):
    def __init__(self, input_col='loan_company_name'):
        self.input_col = input_col
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Create a copy to avoid modifying the original
        df = X.copy()
        
        # Step 1: Basic preprocessing
        df['clean_name_re'] = df[self.input_col].apply(self._preprocess_name)
        
        # Step 2: Phonetic coding
        df['phonetic_code'] = df['clean_name_re'].apply(self._get_phonetic_code)
        df['standardized_name_jellyfish'] = df.groupby('phonetic_code')['clean_name_re'].transform('first')
        
        # Step 3: N-gram hashing
        df['ngram_hash'] = df['standardized_name_jellyfish'].apply(self._ngram_hash)
        df['standardized_name_ngram_hash'] = df.groupby('ngram_hash')['standardized_name_jellyfish'].transform('first')
        
        # Step 4: Aggressive normalization
        df['standardized_name_aggressive_normalize'] = df['standardized_name_ngram_hash'].apply(self._aggressive_normalize)
        
        return df
    
    def _preprocess_name(self, name):
        name = str(name).lower()  # Convert to lowercase
        name = re.sub(r'[^\w\s]', '', name)  # Remove punctuation
        
        # Remove common suffixes
        suffixes = ['inc', 'incorporated', 'corp', 'corporation', 'ltd', 'limited', 'co', 'company', 'plc']
        pattern = r'\b(?:{})\b'.format('|'.join(suffixes))
        name = re.sub(pattern, '', name)
        
        name = ' '.join(name.split())  # Remove extra spaces
        name = name.replace(' ', '')  # Remove spaces between words
        
        return name
    
    def _get_phonetic_code(self, name):
        name = str(name)
        code = jellyfish.metaphone(name)
        return code
    
    def _ngram_hash(self, name, n=3):
        name = str(name).lower()
        name = re.sub(r'[^\w\s]', '', name)
        tokens = name.split()
        ngrams = [''.join(tokens[i:i+n]) for i in range(len(tokens)-n+1)] if len(tokens) >= n else [name]
        combined = ' '.join(ngrams)
        return hashlib.md5(combined.encode()).hexdigest()
    
    def _aggressive_normalize(self, name):
        name = str(name).lower()
        name = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')
        name = re.sub(r'[^\w\s]', '', name)
        name = re.sub(r'\d+', '', name)  # Remove digits
        
        stop_words = ['the', 'and', 'of', 'in', 'a']
        tokens = [word for word in name.split() if word not in stop_words]
        
        suffixes = ['inc', 'incorporated', 'corp', 'corporation', 'company', 'co', 'ltd', 'limited', 'plc']
        tokens = [word for word in tokens if word not in suffixes]
        
        name = ' '.join(sorted(tokens))
        return name.strip().lower()  # Ensure final result is lowercase

# Create the pipeline
def create_company_name_pipeline(input_col='loan_company_name'):
    pipeline = Pipeline([
        ('company_name_standardizer', CompanyNameStandardizer(input_col=input_col))
    ])
    return pipeline

# Example usage
if __name__ == "__main__":
    # Sample data

    
    # Apply the pipeline
    pipeline = create_company_name_pipeline()
    result_df = pipeline.transform(df)
    
    # Display results
    print(result_df[[
        'loan_company_name', 
        'clean_name_re',
        'standardized_name_jellyfish',
        'standardized_name_ngram_hash',
        'standardized_name_aggressive_normalize'
    ]])

                                  loan_company_name  \
0                                               NaN   
1                            Expert Security Agency   
2                            leekie enterprises inc   
3                                    Mangiboa Store   
4                                     Fruits Vendor   
...                                             ...   
64769                                           MCW   
64770                                      LGU Bago   
64771                               Lending Company   
64772                                           HRG   
64773  sanmiguel integrated logistics services inc.   

                              clean_name_re  \
0                                       nan   
1                      expertsecurityagency   
2                         leekieenterprises   
3                             mangiboastore   
4                              fruitsvendor   
...                                     ...   
64769     

In [22]:
result_df.to_csv("Final_cleaned_company_names.csv", index=False)

In [27]:
import pandas as pd
import re
import jellyfish
import hashlib
import unicodedata
from sklearn.base import BaseEstimator, TransformerMixin

class CompanyNameCleaner(BaseEstimator, TransformerMixin):
    def __init__(self, ngram_n=3):
        # You can add parameters here if needed
        self.ngram_n = ngram_n
        pass

    def fit(self, X, y=None):
        # This transformer doesn't learn from the data
        return self

    def transform(self, X):
        X = X.copy()
        
        # Step 1: Preprocess the names
        X['clean_name'] = X['loan_company_name'].apply(self.preprocess_name)
        
        # Step 2: Apply phonetic coding
        X['phonetic_code'] = X['clean_name'].apply(self.get_phonetic_code)
        X['standardized_name_phonetic'] = X.groupby('phonetic_code')['loan_company_name'].transform('first')
        
        # Step 3: Apply n-gram hashing
        X['ngram_hash'] = X['standardized_name_phonetic'].apply(self.ngram_hash)
        X['standardized_name_ngram'] = X.groupby('ngram_hash')['standardized_name_phonetic'].transform('first')
        
        # Step 4: Aggressive normalization
        X['standardized_name_aggressive'] = X['standardized_name_ngram'].apply(self.aggressive_normalize)
        
        # Return the DataFrame with the final cleaned column
        return X

    def preprocess_name(self, name):
        if pd.isnull(name):
            return ''
        name = str(name).lower()
        name = re.sub(r'[^\w\s]', '', name)       # Remove punctuation
        # Remove common suffixes
        suffixes = ['inc', 'incorporated', 'corp', 'corporation', 'ltd', 'limited', 'co', 'company', 'plc']
        pattern = r'\b(?:{})\b'.format('|'.join(suffixes))
        name = re.sub(pattern, '', name)
        name = ''.join(name.split())               # Remove all spaces between names
        return name

    def get_phonetic_code(self, name):
        return jellyfish.metaphone(name)

    def ngram_hash(self, name, n=3):
        name = str(name).lower()
        name = re.sub(r'[^\w\s]', '', name)
        tokens = name.split()
        if len(tokens) < n:
            ngrams = [''.join(tokens)]
        else:
            ngrams = [''.join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]
        combined = ' '.join(ngrams)
        return hashlib.md5(combined.encode()).hexdigest()

    def aggressive_normalize(self, name):
        name = str(name).lower()
        # Remove accents
        name = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')
        name = re.sub(r'[^\w\s]', '', name)        # Remove punctuation
        name = re.sub(r'\d+', '', name)            # Remove digits
        # Remove stop words
        stop_words = ['the', 'and', 'of', 'in', 'a']
        tokens = [word for word in name.split() if word not in stop_words]
        # Remove suffixes
        suffixes = ['inc', 'incorporated', 'corp', 'corporation', 'company', 'co', 'ltd', 'limited', 'plc']
        tokens = [word for word in tokens if word not in suffixes]
        name = ' '.join(sorted(tokens))            # Sort tokens
        return name.strip()


In [29]:
from sklearn.pipeline import Pipeline

# Create the pipeline
pipeline = Pipeline([
    ('company_name_cleaner', CompanyNameCleaner(ngram_n=3))
])

# Assume df is your DataFrame with 'loan_company_name' column
# Apply the pipeline
df_cleaned = pipeline.transform(df)

# View the final standardized names



In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.ensemble import RandomForestClassifier

# # Assuming you have a target variable 'y'
# y = [...]  # Your target variable

# # Create a full pipeline
# full_pipeline = Pipeline([
#     ('company_name_cleaner', CompanyNameCleaner()),
#     ('vectorizer', CountVectorizer()),
#     ('classifier', RandomForestClassifier())
# ])

# # Fit the pipeline
# full_pipeline.fit(df[['loan_company_name']], y)


In [30]:
df_cleaned.head()

,global_mean,smoothing_factor,loan_company_name,freq_encodedcompanyName,category_mean_companyName,category_size_companyName,target_encodedcompanyName,encoded_company_name_group,clean_name,phonetic_code,standardized_name_phonetic,ngram_hash,standardized_name_ngram,standardized_name_aggressive
0,30892.506245,15,NaN,NaN,NaN,NaN,30892.506245,NaN,,,EEI Corporation,89aa7553a2ef81ec9076d21e44b4b0d9,EEI Corporation,eei
1,30892.506245,15,Expert Security Agency,1.0,25000.000000,1.0,30524.224605,30524.224605,expertsecurityagency,EKSPRTSKRTYJNS,Expert Security Agency,4c004b846168151b991c9d36b84d42c8,Expert Security Agency,agency expert security
2,30892.506245,15,leekie enterprises inc,1.0,28000.000000,1.0,30711.724605,30711.724605,leekieenterprises,LKNTRPRSS,leekie enterprises inc,b4ab50eb7e380812afbae53f03334c7b,leekie enterprises inc,enterprises leekie
3,30892.506245,15,Mangiboa Store,1.0,15000.000000,1.0,29899.224605,29899.224605,mangiboastore,MNJBSTR,Mangiboa Store,51250b025291d65f4dac3246c2d961bf,Mangiboa Store,mangiboa store
4,30892.506245,15,Fruits Vendor,3.0,28333.333333,3.0,30465.977426,91397.932279,fruitsvendor,FRTSFNTR,Fruits Vendor,68a04ab84f0a560ea880f44d1e130008,Fruits Vendor,fruits vendor


In [31]:
df_cleaned.to_csv("Final_cleaned_company_names_pipeline.csv", index=False)

In [35]:
df_cleaned[df_cleaned['loan_company_name'].str.contains('accent', na=False)]

,global_mean,smoothing_factor,loan_company_name,freq_encodedcompanyName,category_mean_companyName,category_size_companyName,target_encodedcompanyName,encoded_company_name_group,clean_name,phonetic_code,standardized_name_phonetic,ngram_hash,standardized_name_ngram,standardized_name_aggressive
408,30892.506245,15,Rajoma gifts n homes accents. Inc,1.0,10000.000000,1.0,29586.724605,29586.724605,rajomagiftsnhomesaccents,RJMJFTSNHMSKSNTS,Rajoma gifts n homes accents. Inc,e50b6c4e598d11bc183eaf2884f978f5,Rajoma gifts n homes accents. Inc,accents gifts homes n rajoma
3558,30892.506245,15,accenture inc,3.0,41666.666667,3.0,32688.199648,98064.598945,accenture,AKSNTR,Accenture,e96a479c49f59f0f9e66875d0d856ab6,Accenture,accenture
58328,30892.506245,15,accenture,3.0,25333.333333,3.0,29965.977426,89897.932279,accenture,AKSNTR,Accenture,e96a479c49f59f0f9e66875d0d856ab6,Accenture,accenture
